分块是NLP中进行实体检测的基本技术，对多词序列进行分段和标记，将共同表示某个含义的词分割在一起形成一个单元

对于chunking而言，最有效的信息依据是词的pos tag
最简单的分块方法是自己定义一个模式，使用正则匹配的方法对词序列进行匹配，匹配该模式的词序列划分为一个块
下面是根据pos tag划分名字短语的示例，分块的结果将识别到的NP用()划分在一起

In [ ]:
import nltk

sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"), ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]

grammar = r"""
    NP: {<DT|PP\$>?<JJ>*<NN>}
        {<NNP>+}
"""
cp = nltk.RegexpParser(grammar) # 正则分块器
result = cp.parse(sentence)
print(result)

In [ ]:
from nltk.corpus import brown
for sent in brown.tagged_sents()[:2]:
    tree = cp.parse(sent)
    for subtree in tree.subtrees():
        if subtree.label() == 'NP':
            print(subtree)

NLTK支持Chinking方法，将满足条件的token序列从块中删除从而进行分块（以删除的方式进行分块而不是划分的方式）
简单的示例如下，下面的分块方法里匹配第一个模式的token序列会作为一个NP，匹配第二个模式的token序列会被删除

In [ ]:
grammar = r"""
  NP:
    {<.*>+}          # Chunk everything
    }<VBD|IN>+{      # Chink sequences of VBD and IN
  """
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),
       ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]
cp = nltk.RegexpParser(grammar)
cp.parse(sentence)

NLTK可以对分块器的分块结果进行评估，可以对分块结果的Precision，Recall，F_Meature进行评估，IOB Accuracy表示IOB准确率 可以认为是分块准确率

In [ ]:
from nltk.corpus import conll2000
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
grammar = r"NP: {<[CDJNP].*>+}"
cp = nltk.RegexpParser(grammar)
print(cp.evaluate(test_sents))

通过扩展ChunkParserI接口，可以设计自己的分块器
下面通过Unigram的tag，根据Unigram的词性标注结果进行分块
可以将Unigram扩展到Bigram和N-gram

In [ ]:
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [ ]:
train_sents = conll2000.chunked_sents('train.txt', chunk_types=['NP'])
unigram_chunker = UnigramChunker(train_sents)
print(unigram_chunker.evaluate(test_sents))

句子中token词性的标注无法完全决定句子分块的方式，需要将词的语义内容也作为分块的依据之一
基于这种方式可以使用基于分类器的tagger对句子进行分块
代码示例如下：

In [ ]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    return {"pos": pos, "word": word, "prevpos": prevpos}

In [ ]:
class ConsecutiveNPChunkTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        """对词的tag包含词性标记和分类结果（词义信息）"""
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)


class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [ ]:
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.evaluate(test_sents))

当引入了不同Chunk类型的分块规则后，分块得到的结构是层级结构而非上述示例中的单层结构
NLTK使用Tree（树结构）代表Chunk后的层级结构，通过draw方法可视化
可以通过nltk.chunk.conlltags2tree方法将tag序列转换为Tree 树

In [ ]:
grammar = r"""
  NP: {<DT|JJ|NN.*>+}          # Chunk sequences of DT, JJ, NN
  PP: {<IN><NP>}               # Chunk prepositions followed by NP
  VP: {<VB.*><NP|PP|CLAUSE>+$} # Chunk verbs and their arguments
  CLAUSE: {<NP><VP>}           # Chunk NP, VP
  """
cp = nltk.RegexpParser(grammar)
sentence = [("Mary", "NN"), ("saw", "VBD"), ("the", "DT"), ("cat", "NN"),
    ("sit", "VB"), ("on", "IN"), ("the", "DT"), ("mat", "NN")]
print(cp.parse(sentence))

命名实体是指特定类型的名词短语，如组织、个人、日期等。
命名实体识别要识别文本中的所有命名实体，是一种特殊的分块任务：
+ 识别命名实体的边界
+ 识别命名实体的类型

命名实体识别适用于基于分类器的方法，根据词性标记和词义信息进行命名实体识别
NLTK提供了内置的命名实体识别器ne_chunk进行命名实体识别，
其中binary参数为True将命名实体标记为NE，否则添加类别标签

In [ ]:
sent = nltk.corpus.treebank.tagged_sents()[22]
print(nltk.ne_chunk(sent, binary=True))
print(nltk.ne_chunk(sent, binary=False))

命名实体关系的识别可以通过模式匹配，即识别满足特定模式的两个命名实体
如A in B就是一个可行的模式，体现了两个命名实体之间的包含关系
以下是使用NLTK进行命名实体关系识别的示例，通过nltk.sem.extract_rels识别文本中命名实体之间的关系，通过nltk.sem.rtuple提取元组进行输出

In [ ]:
import re
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
    for rel in nltk.sem.extract_rels('ORG', 'LOC', doc, corpus='ieer', pattern=IN):
        print(nltk.sem.rtuple(rel))